<a href="https://colab.research.google.com/github/awais5597/Facial-monitoring-and-3D-reconstruction/blob/main/3d_mapping_of_faces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Load Image
from google.colab import files
import cv2

# Upload file to Colab
uploaded = files.upload()

# Replace file if it already exists
filename = list(uploaded.keys())[0]
with open(filename, "wb") as f:
    f.write(uploaded[filename])

# Read uploaded image file using OpenCV
img = cv2.imread(filename, cv2.IMREAD_COLOR)



Saving input_image.jpg to input_image (1).jpg


In [ ]:
# Install the required package
!pip install plotly
!pip install open3d
!pip install pip install tensorflow-datasets


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Required libraries
import cv2
import dlib
import numpy as np
from imutils import face_utils
import open3d as o3d
import urllib.request
import plotly.graph_objs as go
import tensorflow as tf

In [ ]:
# Download and extract the shape predictor file
url = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"
file_name = "shape_predictor_68_face_landmarks.dat.bz2"

print(f"Downloading {file_name}...")
urllib.request.urlretrieve(url, file_name)
print("Download complete!")

print("Extracting the downloaded file...")
import bz2
with open("shape_predictor_68_face_landmarks.dat", "wb") as f:
    f.write(bz2.BZ2File("shape_predictor_68_face_landmarks.dat.bz2").read())
print("Extraction complete!")

Download complete!
Extracting the downloaded file...
Extraction complete!


In [ ]:
# Load the image from a URL
def url_to_image(url):
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    return image

In [ ]:
import tensorflow_datasets as tfds

def load_facial_trait_classification_data():
    # Load the CelebA dataset
    (train_data, val_data, test_data), ds_info = tfds.load(
        'celeb_a',
        split=['train', 'validation', 'test'],
        shuffle_files=True,
        with_info=True,
        as_supervised=True
    )

    # Select the trait you want to classify (e.g., "Smiling")
    label_index = ds_info.features['attributes'].keys().index('Smiling')

    # Preprocess the images and labels
    def preprocess(image, attributes):
        image = tf.cast(image, tf.float32) / 255.0
        image = tf.image.resize(image, (32, 32))
        label = tf.cast(attributes[label_index], tf.int64)
        return image, label

    train_data = train_data.map(preprocess)
    val_data = val_data.map(preprocess)
    test_data = test_data.map(preprocess)

    # Convert to NumPy arrays
    train_images, train_labels = zip(*list(train_data.as_numpy_iterator()))
    val_images, val_labels = zip(*list(val_data.as_numpy_iterator()))
    test_images, test_labels = zip(*list(test_data.as_numpy_iterator()))

    return (np.array(train_images), np.array(train_labels)), (np.array(test_images), np.array(test_labels))


In [ ]:
def create_facial_trait_classification_model():
    import numpy as np
    import tensorflow as tf
    from tensorflow.keras import layers
    from tensorflow.keras.models import Sequential

    # Load the facial trait classification dataset
    (train_images, train_labels), (test_images, test_labels) = load_facial_trait_classification_data()

    # Shuffle the data
    perm = np.random.permutation(len(train_images))
    train_images, train_labels = train_images[perm], train_labels[perm]

    # Split the data into training and validation sets
    num_train = int(0.8 * len(train_images))
    val_images, val_labels = train_images[num_train:], train_labels[num_train:]
    train_images, train_labels = train_images[:num_train], train_labels[:num_train]

    # Preprocess the data
    train_images = train_images.astype('float32') / 255.0
    val_images = val_images.astype('float32') / 255.0
    test_images = test_images.astype('float32') / 255.0

    # Convert labels to one-hot encoding
    train_labels = tf.keras.utils.to_categorical(train_labels)
    val_labels = tf.keras.utils.to_categorical(val_labels)
    test_labels = tf.keras.utils.to_categorical(test_labels)

    # Define the CNN model architecture
    model = Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(7, activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Train the model on the facial trait classification dataset
    model.fit(train_images, train_labels, epochs=10, 
              validation_data=(val_images, val_labels))

    # Save the trained model to a file
    model.save('facial_trait_classification_model.h5')

In [ ]:
# Load the trained facial trait classification model
def load_model():
    model = tf.keras.models.load_model('facial_trait_classification_model.h5')


In [ ]:
# Function to pre-process image
def pre_process_image(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    return gray

# Function to detect faces in image and extract landmarks
def detect_faces_and_landmarks(img):
    faces = detector(img)
    all_landmarks = []
    for face in faces:
        landmarks = predictor(img, face)
        landmarks = face_utils.shape_to_np(landmarks)
        all_landmarks.append(landmarks)
    return all_landmarks

# Function to classify facial traits using machine learning
def classify_facial_traits(landmarks):
    features = []
    for landmark in landmarks:
        # Extract facial features from landmarks
        feature = np.array([landmark])
        # Pre-process features for classification
        feature = pre_process_feature(feature)
        # Classify features using the trained model
        prediction = model.predict(feature)
        # Add the predicted label to the list of features
        features.append(prediction)
    return features

# Function to create a 3D point cloud from the 2D landmarks
def create_point_cloud(landmarks, features):
    points = []
    for i, (x, y) in enumerate(landmarks):
        # Add the facial feature as a color to the point cloud
        color = features[i][0]
        points.append([x, y, 0.0, color])
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(np.array(points)[:, :3])
    point_cloud.colors = o3d.utility.Vector3dVector(np.array(points)[:, 3:])
    return point_cloud

In [ ]:
# Create the detector and predictor objects
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [ ]:
# Detect the faces in the image
faces = detector(img)


In [ ]:
# Run the code above to get the 68 facial landmarks
for face in faces:
    landmarks = predictor(img, face)
    landmarks = face_utils.shape_to_np(landmarks)

    # Create a 3D point cloud from the 2D landmarks
    points = []
    for i, (x, y) in enumerate(landmarks):
        points.append([x, y, 0.0])

    # Visualize the 3D model using plotly
    x, y, z = zip(*points)
    scatter3d = go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(size=5, color='red'))
    layout = go.Layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'))
    fig = go.Figure(data=[scatter3d], layout=layout)
    fig.show()